In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_predict

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-mar-2021/train.csv").drop(columns = "id")
test = pd.read_csv("../input/tabular-playground-series-mar-2021/test.csv")

In [ ]:
train = train_df.iloc[:, :-1]
train

In [ ]:
train_y = train_df.iloc[:, -1]
train_y

In [ ]:
cat = []
cont = []
for i in train.columns:
    if train[i].dtype == object:
        cat.append(i)
    else:
        cont.append(i)

In [ ]:
print(cat, cont)

In [ ]:
train_cat = train.drop(columns = cont)
train_cont = train.drop(columns = cat)

In [ ]:
print(train_cat.columns, train_cont.columns)

# Categorical

In [ ]:
def get_class(col):
    cat_df = train_cat.iloc[:, 1:].groupby(col,as_index=True)[[col]].count().rename(columns={col:'count'}).sort_values(by='count',ascending=False)
    cat_df = cat_df.reset_index()
    cat_df['percent'] = (cat_df['count']*100)/(cat_df['count'].sum())
    cat_df['cmsum'] = cat_df['percent'].cumsum()
    return list(cat_df[cat_df['cmsum']<96][col])

In [ ]:
cat2_class = get_class('cat2')
cat3_class = get_class('cat3')
cat4_class = get_class('cat4')
cat5_class = get_class('cat5')
cat6_class = get_class('cat6')
cat7_class = get_class('cat7')
cat8_class = get_class('cat8')
cat9_class = get_class('cat9')
cat10_class = get_class('cat10')

In [ ]:
train_cat['cat2'] = train_cat['cat2'].apply(lambda x: x if x in cat2_class else 'cat2')
train_cat['cat3'] = train_cat['cat3'].apply(lambda x: x if x in cat3_class else 'cat3')
train_cat['cat4'] = train_cat['cat4'].apply(lambda x: x if x in cat4_class else 'cat4')
train_cat['cat5'] = train_cat['cat5'].apply(lambda x: x if x in cat5_class else 'cat5')
train_cat['cat6'] = train_cat['cat6'].apply(lambda x: x if x in cat6_class else 'cat6')
train_cat['cat7'] = train_cat['cat7'].apply(lambda x: x if x in cat7_class else 'cat7')
train_cat['cat8'] = train_cat['cat8'].apply(lambda x: x if x in cat8_class else 'cat8')
train_cat['cat9'] = train_cat['cat9'].apply(lambda x: x if x in cat9_class else 'cat9')
train_cat['cat10'] = train_cat['cat10'].apply(lambda x: x if x in cat10_class else 'cat10')

In [ ]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
cat_new = ohe.fit_transform(train_cat)
cat_new = pd.DataFrame(cat_new)
cat_new

# Numerical

In [ ]:
train_cont = train_cont.drop(columns = ['cont1', 'cont7'])
train_cont

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
cont_new = sc.fit_transform(train_cont)
cont_new = pd.DataFrame(cont_new)

In [ ]:
cont_new

In [ ]:
cont_new.columns = [253,254,255,256,257,258,259,260,261]
cont_new

In [ ]:
train_new = pd.concat([cat_new, cont_new], axis = 1)
train_new.head()

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 0.001)
# cv_la = cross_validate(lasso,train_new, train_y,cv = 10,scoring = 'roc_auc')
# cv_la['test_score'].mean()
lasso.fit(train_new, train_y)

In [ ]:
n = lasso.coef_
n

In [ ]:
n.shape

In [ ]:
df = pd.DataFrame(n)
df.insert(0, 'New_ID', range(0, 0 + len(df)))
df

In [ ]:
df.iloc[:, -1] < 0

In [ ]:
df['keep ?'] = abs(df.iloc[:, 1]) > 0


In [ ]:
keep = df[df["keep ?"] == True]
keep

In [ ]:
keep.info()

In [ ]:
keep.iloc[:, 0]


In [ ]:
train_new.columns

In [ ]:
lasso_train = train_new[keep.iloc[:, 0]]
lasso_train

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 42)
cv_lr = cross_validate(RF,lasso_train, train_y,cv = 10,scoring="roc_auc")

print(cv_lr)
print(cv_lr['test_score'].mean())

# Test

In [ ]:
id_test = test.loc[:, "id"]
id_test

In [ ]:
test.drop(columns = "id", inplace = True)

In [ ]:
test.head()

In [ ]:
# For Test
cat2 = []
cont2 = []
for i in test.columns:
    if test[i].dtype == object:
        cat2.append(i)
    else:
        cont2.append(i)

In [ ]:
test_cat = test.drop(columns = cont2)
test_cont = test.drop(columns = cat2)

In [ ]:
print(test_cat.columns, test_cont.columns)

In [ ]:
def get_class_test(col):
    cat_df = test_cat.iloc[:, 1:].groupby(col,as_index=True)[[col]].count().rename(columns={col:'count'}).sort_values(by='count',ascending=False)
    cat_df = cat_df.reset_index()
    cat_df['percent'] = (cat_df['count']*100)/(cat_df['count'].sum())
    cat_df['cmsum'] = cat_df['percent'].cumsum()
    return list(cat_df[cat_df['cmsum']<96][col])

In [ ]:
cat2_class = get_class_test('cat2')
cat3_class = get_class_test('cat3')
cat4_class = get_class_test('cat4')
cat5_class = get_class_test('cat5')
cat6_class = get_class_test('cat6')
cat7_class = get_class_test('cat7')
cat8_class = get_class_test('cat8')
cat9_class = get_class_test('cat9')
cat10_class = get_class_test('cat10')

In [ ]:
test_cat['cat2'] = test_cat['cat2'].apply(lambda x: x if x in cat2_class else 'cat2')
test_cat['cat3'] = test_cat['cat3'].apply(lambda x: x if x in cat3_class else 'cat3')
test_cat['cat4'] = test_cat['cat4'].apply(lambda x: x if x in cat4_class else 'cat4')
test_cat['cat5'] = test_cat['cat5'].apply(lambda x: x if x in cat5_class else 'cat5')
test_cat['cat6'] = test_cat['cat6'].apply(lambda x: x if x in cat6_class else 'cat6')
test_cat['cat7'] = test_cat['cat7'].apply(lambda x: x if x in cat7_class else 'cat7')
test_cat['cat8'] = test_cat['cat8'].apply(lambda x: x if x in cat8_class else 'cat8')
test_cat['cat9'] = test_cat['cat9'].apply(lambda x: x if x in cat9_class else 'cat9')
test_cat['cat10'] = test_cat['cat10'].apply(lambda x: x if x in cat10_class else 'cat10')


In [ ]:
t_cat_new = ohe.fit_transform(test_cat)
t_cat_new = pd.DataFrame(t_cat_new)
t_cat_new

In [ ]:
test_cont = test_cont.drop(columns = ['cont1', 'cont7'])
test_cont

In [ ]:
t_cont_new = sc.transform(test_cont)
t_cont_new = pd.DataFrame(t_cont_new)
t_cont_new

In [ ]:
t_cont_new.columns = [253,254,255,256,257,258,259,260,261]
t_cont_new

In [ ]:
test_new = pd.concat([t_cat_new, t_cont_new], axis = 1)
test_new.head()

In [ ]:
lasso_test = test_new[keep.iloc[:, 0]]
lasso_test

In [ ]:
final_RF = RF.fit(lasso_train, train_y)
final = final_RF.predict_proba(lasso_test)
final_sub = final[:, 1]
final_sub

In [ ]:
sub = pd.DataFrame({'id':id_test,'target':final_sub})
sub

In [ ]:
sub.to_csv('submission.csv', index = False)